In [37]:
'''THis block of code pre processes the data'''
from nltk.stem import WordNetLemmatizer
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize,sent_tokenize

stop_words = set(stopwords.words("english") + ['data', 'mesh', 'impact', 'question', 'domain', 'impact', 'practices', 'success', 'motivation', 'ieee', 'xxx', 'xxxx', 'xx', 'discussion', 'www', 'com', 'sciencedirect', 'online','journal','motivational','factor','lectures','author','interview','interveee','thesis'])
                                            


file_path = 'dataset/Data_mesh_csv.csv'

# Read the TSV file and convert it to a DataFrame
df = pd.read_csv(file_path, delimiter=',')



In [38]:
def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE) #remove white space
    RE_TAGS = re.compile(r"<[^>]+>") #removes tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE) #keep only asscii character
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", str(text))
    text = re.sub(RE_ASCII, " ", str(text))
    text = re.sub(RE_SINGLECHAR, " ", str(text))
    text = re.sub(RE_WSPACE, " ", str(text))

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [lemmatizer.lemmatize(w) for w in words_tokens_lower if w not in stop_words]
        


    text_clean = " ".join(words_filtered)
    return text_clean
def drop_nan(df):
    
    df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True)
    
    return df


In [39]:
pdf_data = df.copy()
drop_nan(pdf_data)
pdf_data['pdf_content'] = pdf_data['pdf_content'].apply(clean_text)
pdf_data

,pdf_name,pdf_content
0,0670.pdf,introducing paradigm smart city platform desig...
1,1-s2.0-S1877050921022365-main.pdf,available procedia computer science author pub...
2,1-s2.0-S1877050923006099-main.pdf,available procedia computer science author pub...
3,2302.01713.pdf,best avoid mess jan bode ibm germany niklas kü...
4,2304.01062.pdf,systematic gray literature review abel goedege...
5,978-1-4842-9253-2.pdf,fabric approach ai guide ai based cataloging g...
6,978-3-031-12423-5_7.pdf,cok survey privacy challenge relation mesh nik...
7,978-3-031-36118-0.pdf,lecture note engineering communication technol...
8,978-3-031-39847-6_1.pdf,integration revitalized warehouse lake robert ...
9,978-3-031-45021-1_23.pdf,converging microservice principle uni ed logic...


In [40]:
csv_filename = 'Data_mesh_cleaned.csv'

# Save the DataFrame to a CSV file
pdf_data.to_csv(csv_filename, index=False)